In [ ]:
import selenium
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.by import By

import chromedriver_autoinstaller #자동으로 크롬드라이버 설치 (크롬쓰는게 좋음)
from tqdm import tqdm  # for문 진행상황 파악

import pandas as pd
import numpy as np
import time
import os
import sys

In [ ]:
# 크롬 드라이버를 '현재 작업 폴더'에 설치    #install 설치
chromedriver_autoinstaller.install(cwd=True)

driver = webdriver.Chrome()   # driver : 크롬 브라우저

driver.get('https://ai-dev.tistory.com/1')    #https://ai-dev.tistory.com/1
time.sleep(2)  # 2초간 정지

In [ ]:
----------------------------------------------------------------------------------------------------

In [5]:
# 글 제목   selector:선택자
selector = "#content > div > div.hgroup > h1"
title = driver.find_element(By.CSS_SELECTOR, selector).text
title

'크롤링의 세계에 오신 것을 환영합니다.'

In [6]:
# 글쓴이   웹사이트 복잡해지면 쓰기 안좋음 걍 위 방식으로 암기
# class=".author" 활용  @@@클래스는 점찍어야
driver.find_element(By.CSS_SELECTOR, ".author").text

'로스카츠'

In [7]:
# copy selector 로 복사해서 selector 사용 
selector = "#content > div > div.hgroup > div.post-meta > span.author"
author = driver.find_element(By.CSS_SELECTOR, selector).text
author

'로스카츠'

In [8]:
# 날짜
selector = '.date'
driver.find_element(By.CSS_SELECTOR, selector).text

'2021. 1. 21. 14:06'

In [9]:
selector = '.tt_article_useless_p_margin.contents_style'
driver.find_element(By.CSS_SELECTOR, selector).text

'Hello, world!'

In [ ]:
# <실습> 티스토리 셀레늄 크롤링 https://ai-dev.tistory.com/1
# 블로그 제목 "인공지능 개발의 모든 것"
# 좋아요 수
# 댓글 수
# 댓글 -> 리스트로 저장
# 댓글 글쓴이 -> 리스트로 저장
# 댓글 날짜 -> 리스트로 저장
# 댓글 데이터프레임으로 변환

In [13]:
# 블로그 제목 "인공지능 개발의 모든 것"
selector ='.tt_tit_cont'
blog_title = driver.find_element(By.CSS_SELECTOR, selector).text
blog_title

'인공지능 개발의 모든 것'

In [16]:
'''
이건 CSS에서 “자식 선택자” 의미야.
👉 해석하면
.txt_like 안에 있는 uoc-count 태그 를 찾겠다는 뜻
근데 uoc-count는 태그가 아니라 클래스잖아.
✅ 클래스 2개를 동시에 가진 요소는 이렇게 써야 해
'''
# 좋아요 수
selector ='.txt_like.uoc-count'
like = int(driver.find_element(By.CSS_SELECTOR, selector).text)
like

50

In [19]:
# 댓글 수
selector ='.tt_num_g'
review_cnt = int(driver.find_element(By.CSS_SELECTOR, selector).text)
review_cnt

17

In [23]:
# 댓글 -> 리스트로 저장  #tt_desc 클래스는 여러게 있을 수 있다 카피셀렉터 안짜르고 그대로 다가져오면됨
selector ='#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child(6)'
review_raw = driver.find_element(By.CSS_SELECTOR, selector).text
review_raw

'김모이\n아이시떼루!!!!\n2023. 10. 17. 14:07\n답글\n더보기'

In [24]:
review_raw.split('\n')[1]

'아이시떼루!!!!'

In [25]:
# 댓글 글쓴이 -> 리스트로 저장
review_raw.split('\n')[0]

'김모이'

In [26]:
# 댓글 날짜 -> 리스트로 저장
review_raw.split('\n')[2]

'2023. 10. 17. 14:07'

In [32]:
# 댓글 -> 리스트로 저장  #tt_desc 클래스는 여러게 있을 수 있다 한 번 더   카피셀렉터 안짜르고 그대로 다가져오면됨
selector ='#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child(1) > div > div.tt-box-content > div.tt-wrap-desc > p'
review_raw_1 = driver.find_element(By.CSS_SELECTOR, selector).text
review_raw_1

'이천주'

# for 문 기본 첫줄 작성하고 위 내용 복붙 그대로라 외우기 편할듯 참고 

In [38]:
#리뷰수 변수 만들어 뒀으니 변수에 +1 (review_cnt+1)해서 사용  그냥 숫자알연 댓글17개 18이라고 적어도됨
review_list = []

for i in range(1,17):  #1~16 댓글 수 16개(대댓글 1개 따로 수집해야함)
    selector =f"#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child({i}) > div > div.tt-box-content > div.tt-wrap-desc > p"
    review_raw_1 = driver.find_element(By.CSS_SELECTOR, selector).text
    review_list.append(review_raw_1)
review_list   

['이천주',
 '이천주1',
 '이천주2',
 '정수경',
 '정수경2',
 '아이시떼루!!!!',
 '새싹',
 '새싹2',
 '새싹3',
 '새싹4',
 '왹져',
 '내공냠냠',
 '내공좀',
 'dada',
 '헤헤 방가방가',
 '# Hello World']

In [49]:
#글쓴이
selector ='#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child(1) > div > div.tt-box-content > div.tt-box-meta > a'
review_raw_2 = driver.find_element(By.CSS_SELECTOR, selector).text
review_raw_2

'이천주'

In [51]:
#전체 댓글 글쓴이 반복문으로 수집 후 리스트로 저장
review_name_list = []

for i in range(1,17):  
    selector = f"#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child({i}) > div > div.tt-box-content > div.tt-box-meta > a"
    review_raw_2 = driver.find_element(By.CSS_SELECTOR, selector).text
    review_raw_2
    review_name_list.append(review_raw_2)
review_name_list

['이천주',
 '이천주1',
 '이천주2',
 '정수경',
 '정수경2',
 '김모이',
 '새싹',
 '새싹2',
 '새싹3',
 '새싹4',
 '왹져',
 '네이버블로그짱',
 '네이버지식인',
 '111',
 '크롤링1',
 'dev-tyler']

In [52]:
# 댓글 날짜 -> 리스트로 저장
selector ='#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child(1) > div > div.tt-box-content > div.tt-wrap-info > span.tt_date'
driver.find_element(By.CSS_SELECTOR, selector).text

'2023. 4. 26. 16:09'

In [54]:
review_time_list = []

for i in range(1,17):
    selector =f'#entry1Comment > div > div > div > div.tt-area-reply > ul > li:nth-child({i}) > div > div.tt-box-content > div.tt-wrap-info > span.tt_date'
    review_time = driver.find_element(By.CSS_SELECTOR, selector).text
    review_time_list.append(review_time)
review_time_list

['2023. 4. 26. 16:09',
 '2023. 4. 26. 16:10',
 '2023. 6. 26. 20:21',
 '2023. 10. 17. 13:53',
 '2023. 10. 17. 13:53',
 '2023. 10. 17. 14:07',
 '2025. 9. 16. 16:45',
 '2025. 9. 16. 16:45',
 '2025. 9. 18. 13:56',
 '2025. 9. 18. 13:57',
 '2025. 9. 18. 14:20',
 '2025. 9. 18. 14:20',
 '2025. 9. 18. 14:21',
 '2025. 9. 18. 14:35',
 '2025. 9. 19. 19:36',
 '2026. 1. 5. 10:39']

In [55]:
#댓글,댓글 글쓴이.댓글 날짜 =>데이터프레임으로 저장

df = pd.DataFrame({'댓글':review_list, '댓글러':review_name_list, '날짜':review_time_list})
df

,댓글,댓글러,날짜
0,이천주,이천주,2023. 4. 26. 16:09
1,이천주1,이천주1,2023. 4. 26. 16:10
2,이천주2,이천주2,2023. 6. 26. 20:21
3,정수경,정수경,2023. 10. 17. 13:53
4,정수경2,정수경2,2023. 10. 17. 13:53
5,아이시떼루!!!!,김모이,2023. 10. 17. 14:07
6,새싹,새싹,2025. 9. 16. 16:45
7,새싹2,새싹2,2025. 9. 16. 16:45
8,새싹3,새싹3,2025. 9. 18. 13:56
9,새싹4,새싹4,2025. 9. 18. 13:57


In [ ]:
# 댓글 데이터프레임으로 변환